<a href="https://colab.research.google.com/github/joseluisgonzalezBA/joseluisgonzalezBA/blob/main/c240xx_Grupo_JLG_TP_P2_Informe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codo a Codo 2024 - 1º Cuatrimestre - TPGO Parte II - (Solucion Jose Luis Gonzalez)

## Funciones utilizadas en el informe

In [ ]:
# Muestra un texto subrayado con guiones
def subrayar(texto):
  print(texto)
  print('-' * len(texto))

# Muestra un texto enmarcado
def enmarcar(texto):
  ancho = len(texto) + 2
  print(f"+{'=' * ancho}+")
  print(f"| {texto} |")
  print(f"+{'=' * ancho}+")


## Lectura de los juegos de datos recibidos

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
url_gdrive = 'https://drive.google.com/uc?id='

### distributors_profile

In [ ]:
# Lectura del archivo 'distributors_profile.csv'
link_distributors = 'https://drive.google.com/file/d/1v_N5_lMWxM9xgGUjrGQsUCjp_CDxNyrr/view?usp=drive_link'
id_distributors = link_distributors.split('/')[5]
df_distributors = pd.read_csv(url_gdrive + id_distributors, thousands='.') # Correcion del separador de miles '.'

# Filtrado de las filas con id = NaN
df_distributors = df_distributors[df_distributors.id.notna()]

# Conversion del campo 'id' a tipo entero
df_distributors.id = df_distributors.id.astype(int)

# Uso del campo 'id' como index del Dataframe
# df_distributors.set_index('id',inplace=True)
# df_distributors.index.name = None

# Reindexacion
df_distributors.reset_index(drop=True, inplace=True)

enmarcar('Fuente de datos "df_distributors"')
print()

display(df_distributors.head(3),df_distributors.tail(3))
print()

subrayar('Tipos de datos de las columnas:')
print(df_distributors.dtypes)
print()

subrayar('Filas / Columnas')
df_distributors.shape

### exports_to_Paraguay

In [ ]:
# Lectura del archivo 'exports_to_Paraguay.csv'
link_exports = 'https://drive.google.com/file/d/1veTCHiz13WO_VW0o9CLEK4tlEamiark2/view?usp=sharing'
id_exports = link_exports.split('/')[5]
df_exports = pd.read_csv(url_gdrive + id_exports, thousands='.') # Correcion del separador de miles '.'

# Eliminacion de las ultimas 3 columnas vacias
df_exports = df_exports[df_exports.columns[:-3]]

# Correccion del tipo de dato de todas las columnas $$$
for col in df_exports.columns[1:]:
  df_exports[col] = df_exports[col].str.replace('$','')
  df_exports[col] = df_exports[col].str.replace('.','')
  df_exports[col] = df_exports[col].str.replace(',','.')
  df_exports[col] = df_exports[col].astype(float)

# Eliminacion de filas duplicadas
df_exports.drop_duplicates(inplace=True)

# Reindexacion
df_exports.reset_index(drop=True, inplace=True)

enmarcar('Fuente de datos "df_exports"')
print()

display(df_exports.head(3),df_exports.tail(3))
print()

subrayar('Tipos de datos de las columnas:')
print(df_exports.dtypes)
print()

subrayar('Filas / Columnas')
df_exports.shape

### locations_profiles

In [ ]:
# Lectura del archivo 'locations_profiles.csv'
link_locations = 'https://drive.google.com/file/d/1vbPc1RmjfnFQpIUDWKAcGr8jSvkziFrp/view?usp=sharing'
id_locations = link_locations.split('/')[5]
df_locations = pd.read_csv(url_gdrive + id_locations)

# Filtrado de localidades sin distribuidores
# Filtrado de las filas con id = NaN o id < 1
df_locations = df_locations[(df_locations.id.notna()) & (df_locations.id > 0)]

# Conversion del campo 'id' a tipo entero
df_locations.id = df_locations.id.astype(int)

# Reindexacion
df_locations.reset_index(drop=True, inplace=True)

enmarcar('Fuente de datos "df_locations"')
print()

display(df_locations.head(3), df_locations.tail(3))
print()

subrayar('Tipos de datos de las columnas:')
print(df_locations.dtypes)
print()

subrayar('Filas / Columnas')
df_locations.shape

### sales_in_Paraguay

In [ ]:
# Lectura del archivo 'sales_in_Paraguay.xlsx'
link_sales = 'https://docs.google.com/spreadsheets/d/1vZB6mc9JD5PId3mIWYNMEFOTDjfJxsll/edit?usp=sharing&ouid=112935633353329215772&rtpof=true&sd=true'
id_sales = link_sales.split('/')[5]
df_sales = pd.read_excel(url_gdrive + id_sales, skipfooter=1) # No lee la ultima fila de "totales"

# Conversion del campo 'distributor' a tipo entero
df_sales.distributor = df_sales.distributor.astype(int)

# Correccion del tipo de dato de todas las columnas $$$
for col in df_sales.columns[1:]:
  df_sales[col] = df_sales[col].str.replace('$','')
  df_sales[col] = df_sales[col].str.replace('.','')
  df_sales[col] = df_sales[col].str.replace(',','.')
  df_sales[col] = df_sales[col].astype(float)

enmarcar('Fuente de datos "df_sales"')
print()

display(df_sales.head(3), df_sales.tail(3))
print()

subrayar('Tipos de datos de las columnas:')
print(df_sales.dtypes)
print()

subrayar('Filas / Columnas')
df_sales.shape

## Depuracion de las fuentes de datos

In [ ]:
# Analisis de la coincidencia de distribuidores que importan y venden en Paraguay
exports_dist = df_exports.distributor
sales_dist = df_sales.distributor
interseccion_dist = exports_dist.isin(sales_dist)
subrayar('Nº de distribuidores que coinciden en "exports" y "sales":')
print(interseccion_dist.size)
print()
print('Como hay una coincidencia total se asume que ambas fuentes son coherentes')

## Analisis de las fuentes de datos

### Analisis del balance de stock y demanda (sobrestock)

In [ ]:
enmarcar('Analisis del sobrestock en Paraguay')
print()

# Totales x producto en miles de millones
exports_totals_milM = df_exports[df_exports.columns[1:]].sum() / 1e9
sales_totals_milM = df_sales[df_sales.columns[1:]].sum() / 1e9

subrayar('Total de importaciones en Paraguay:')
print(f'$ {exports_totals_milM.sum():.2f} miles de millones\n') # Total de importaciones en miles de millones

subrayar('Total de ventas en Paraguay:')
print(f'$ {sales_totals_milM.sum():.2f} miles de millones\n') # Total de ventas en miles de millones

subrayar('Sobrestock en Paraguay:')
print(f'$ {(exports_totals_milM.sum() - sales_totals_milM.sum()):.2f} miles de millones\n')

print('DIAGNOSTICO: En Paraguay hubo un sobrestock superior a los $ 9 mil millones en el cuatrimestre analizado. ')

### Analisis del sobrestock por producto

In [ ]:
enmarcar('Analisis del sobrestock en Paraguay por producto')
print()

sstock_M = (exports_totals_milM - sales_totals_milM) * 1e3
subrayar('Sobrestock por producto (en millones:)')

for i,p in sstock_M.items():
  print(f'{i:10}: $ {p:8.2f} M {"(*)" if p > 0 else ""}')
print()

subrayar('Productos con sobrestock positivo (en millones:)')
for i,p in sstock_M[sstock_M >= 0].items():
  print(f'{i:10}: $ {p:8.2f} M')
print()

print('DIAGNOSTICO: En Paraguay hubo importaciones excesivas de 2 productos (*) con poco nivel de ventas.')

### Analisis del contexto del sobrestock de acero y ladrillo (localidades)

In [ ]:
enmarcar('Analisis del contexto del sobrestock en Paraguay')
print()
print('Total de localidades con distribuidores:',df_locations.index.size)
print()

localidades_steel = df_locations.activities.str.contains('Steel', case=False)
localidades_brick = df_locations.activities.str.contains('Brick', case=False)

subrayar("Localidades con actividades relacionadas al 'acero'")
display(df_locations[localidades_steel])
print()
print('Total de localidades:',df_locations[localidades_steel].index.size)
print()

subrayar("Localidades con actividades relacionadas al 'ladrillo'")
display(df_locations[localidades_brick])
print()
print('Total de localidades:',df_locations[localidades_brick].index.size)
print()

print('DIAGNOSTICO: En Paraguay comercian acero o ladrillo solamente 2 localidades de un total de 10 con distribuidores.')

### Analisis del contexto del sobrestock de acero y ladrillo (distribuidores)

In [ ]:
# Ventas de 'acero' y 'ladrillo' por distribuidor
syb_sales = df_sales[['distributor','Steel','Brick']].rename(columns={'Steel':'steel_sales','Brick':'brick_sales'})

# Importaciones de 'acero' y 'ladrillo' por distribuidor
syb_imports = df_exports[['distributor','Steel','Brick']].rename(columns={'Steel':'steel_imports','Brick':'brick_imports'})

# Consolidacion de ventas e importaciones de acero y ladrillo
syb = syb_sales.merge(syb_imports, on='distributor')
syb.sort_values('distributor', inplace=True)

# Agregado de columnas de balance para acero y ladrillo
syb['steel_sstock'] = syb.steel_imports - syb.steel_sales
syb['brick_sstock'] = syb.brick_imports - syb.brick_sales

enmarcar('Sobrestock de "acero" y "ladrillo" por distribuidor')
print()

subrayar('| id_distributor | steel_sstock | brick_sstock |')
for i,d in syb.iterrows():
  print(f'| {d.distributor:14.0f} | $ {d.steel_sstock/1e6:8.2f} M | $ {d.brick_sstock/1e6:8.2f} M |')

steel_sstock_M = syb.steel_sstock.sum()/1e6
brick_sstock_M = syb.brick_sstock.sum()/1e6
print('-' * 63)
print(f'| {"Totales":14} | $ {steel_sstock_M:8.2f} M | $ {brick_sstock_M:8.2f} M | $ {steel_sstock_M + brick_sstock_M:8.2f} M |')
print('-' * 63)
print()

print('CONCLUSION: todos los distribuidores tienen sobrestock para los 2 productos analizados totalizando casi $ 19 mil millones.')

In [ ]:
# Top 5 sobrestocks de "acero"
sstock_steel = syb.sort_values('steel_sstock', ascending=False)[:5]
enmarcar('Top 5 sobrestocks de "acero" por distribuidor')
subrayar('| id_distributor | steel_sstock |')
for i,d in sstock_steel.iterrows():
  print(f'| {d.distributor:14.0f} | $ {d.steel_sstock/1e6:8.2f} M |')
print()

gra = sns.barplot(data=sstock_steel, x='distributor', y='steel_sstock', color='tomato', order=sstock_steel.distributor)
plt.yscale('log')
gra.set_title('Top 5 sobrestocks de acero (en millones)')
yticks = gra.get_yticks()
gra.set_yticks(yticks)
gra.set_yticklabels([f'{x:.0f} M' for x in yticks / 1e6])
gra.bar_label(gra.containers[0], labels=[f'${art.get_height() / 1e6:.2f} M' for art in gra.containers[0]], label_type='edge')
plt.show()

In [ ]:
# Top 5 sobrestocks de "ladrillo"
sstock_brick = syb.sort_values('brick_sstock', ascending=False)[:5]
enmarcar('Top 5 sobrestocks de "ladrillo" por distribuidor')
subrayar('| id_distributor | brick_sstock |')
for i,d in sstock_brick.iterrows():
  print(f'| {d.distributor:14.0f} | $ {d.brick_sstock/1e6:8.2f} M |')
print()

gra = sns.barplot(data=sstock_brick, x='distributor', y='brick_sstock', order=sstock_brick.distributor, color='tomato')
gra.set_title('Top 5 sobrestocks de ladrillo (en millones)')
yticks = gra.get_yticks()
gra.set_yticks(yticks)
gra.set_yticklabels([f'{x:.0f} M' for x in yticks / 1e6])
gra.bar_label(gra.containers[0], labels=[f'${art.get_height() / 1e6:.2f} M' for art in gra.containers[0]], label_type='edge')
plt.show()

## Conclusiones finales

La baja performance en cuanto a ganancias netas en Paraguay, se vio afectada negativamente por multiples motivos relacionados:

- La ubicacion de los distribuidores en localidades que no tienen el perfil comercial de CMM.
- La falta de distribuidores en ciudades importantes del pais.
- El sobrestock de los materiales `acero` y `ladrillo` en cifras muy altas en todos los distribuidores en general.
- 4 distribuidores tuvieron sobrestocks altisimos de `acero` y `ladrillo`.